In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import csv

In [ ]:
# Project names obtained from the reposiroty crawling in GitHub with the apacheGitHub notebook.
repos_github = pd.read_csv("/home/mikel/Desktop/project1/cleanRepos070223.csv")
github_fullNames = list(repos_github['full_name'])

In [ ]:
'''
PARAM: Receives the full name of the project repository "apache/{repo}"
RETURN: The GitHub project ID of the project.
'''
def getProjectId(projectfullname):

    name = projectfullname + ".csv"

    projectFile = pd.read_csv('/home/mikel/Desktop/project1/githubIssueTables(UnixEpochs)/' + name)

    return projectFile['project_id'][0]

In [ ]:
personal_token = ""
token = os.getenv('GITHUB_TOKEN', personal_token)
headers = {'Authorization': f'token {token}'}

'''
PARAM: projectFullName - Receives the full name of the project repository "apache/{repo}"
       tableToUpdate - Receives the partial location of the project file regarding issue data and
                       updates its information

RETURN: Rerites the file in question.
'''
def makeIssueTable(projectFullName,tableToUpdate):

    # API call url
    theIssueQuery = f"https://api.github.com/repos/{projectFullName}/issues"
    params = {'per_page': 100, 'state': 'all'}
    page = 1

    projectissuedataitems = []
    # Headers of the file to be updated
    issue_feature_list = ['project_id', 'issue_id', 'number', 'user_login', 'user_type', 'state', 'locked', 'created_at', 'updated_at', 'closed_at', 'active_time', 'title', 'body', 'milestone_url', 'comments', 'pull_request']

    with open(f"/home/mikel/Desktop/project1/{tableToUpdate}", 'w', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow([x for x in issue_feature_list])
        file.close()

    while 1 == 1:
        params['page'] = page
        print(projectFullName+' '+'page '+str(page))
        theResult = requests.get(theIssueQuery, headers=headers, params=params)
        theItemListPerPage = theResult.json()

        project_id = getProjectId(projectFullName)

        if len(theItemListPerPage) == 0: # If there's nothing in request.
            break
        else:
            for item in theItemListPerPage:
                issueItem = {}
                issueItem['project_id'] = project_id
                issueItem['issue_id'] = item['id']
                issueItem['number'] = item['number']
                issueItem['user_login'] = item['user']['login']
                issueItem['user_type'] = item['user']['type']
                issueItem['state'] = item['state']
                issueItem['locked'] = item['locked']

                # pd.to_datetime() -> converts a scalar, array-like, Series or DataFrame/dict-like to a pandas datetime object. (utc -> TRUE the function always returns a timezone-aware UTC-localized Timestamp)
                # pd.timestamp() -> It’s the type used for the entries that make up a DatetimeIndex, and other timeseries oriented data structures in pandas.
                # pd.tz_localize() -> Localize tz-naive index of a Series or DataFrame to target time zone.
                # // -> Floor division
                # pd.Timedelta() -> Represents a duration, the difference between two dates or times.
                issueItem['created_at'] = (pd.to_datetime(item['created_at'], utc=True)- pd.Timestamp("1970-01-01").tz_localize('UTC'))// pd.Timedelta('1s')
                if item['updated_at'] == None:
                    issueItem['updated_at'] = np.NaN
                else:
                    issueItem['updated_at'] = (pd.to_datetime(item['updated_at'], utc=True)- pd.Timestamp("1970-01-01").tz_localize('UTC'))// pd.Timedelta('1s')
                if item['closed_at'] == None:
                    issueItem['closed_at'] = np.NaN
                else:
                    issueItem['closed_at'] = (pd.to_datetime(item['closed_at'], utc=True)- pd.Timestamp("1970-01-01").tz_localize('UTC'))// pd.Timedelta('1s')
                if issueItem['state'] == 'closed':
                    issueItem['active_time'] = issueItem['closed_at'] - issueItem['created_at']
                else:
                    if not issueItem['updated_at']:
                        issueItem['active_time'] = 0
                    else:
                        issueItem['active_time'] = issueItem['updated_at'] - issueItem['created_at']
                issueItem['title'] = item['title']
                issueItem['body'] = item['body']
                if item['milestone'] == None:
                    issueItem['milestone_url'] = False
                else:
                    issueItem['milestone_url'] = True
                issueItem['comments'] = item['comments']
                if 'pull_request' in list(item.keys()):
                    issueItem['pull_request'] = True
                else:
                    issueItem['pull_request'] = False
                projectissuedataitems.append(issueItem)
                with open(f"/home/mikel/Desktop/project1/{tableToUpdate}", 'a', encoding='utf-8') as file:
                    writer = csv.writer(file, delimiter=',')
                    writer.writerow([issueItem[x] for x in issue_feature_list])
            page += 1

In [ ]:
for i in range(len(github_fullNames)):

    tableToUpdate = f"githubIssueTables(UnixEpochs)/{github_fullNames[i][7:]}.csv"
    makeIssueTable(github_fullNames[i],tableToUpdate)